# 🟢 Real-Time Object Detection with YOLOv8 COCO and Intel RealSense D435

This notebook demonstrates real-time object detection using the YOLOv8 COCO model and Intel RealSense D435 camera.

## 1. Install Required Packages

Run this cell to install all necessary Python packages for RealSense and YOLOv8 detection.

In [1]:
# Install required packages (run once)
%pip install pyrealsense2 opencv-python ultralytics numpy matplotlib

Note: you may need to restart the kernel to use updated packages.


## 2. Import Libraries

Import all necessary libraries for RealSense, YOLOv8, and visualization.

In [2]:
import pyrealsense2 as rs
import numpy as np
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
import time
import torch

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


## 3. Verify RealSense Camera Connection

Check if the Intel RealSense D435 camera is connected and print device information.

In [3]:
# Check for connected RealSense devices
ctx = rs.context()
devices = ctx.query_devices()

if len(devices) == 0:
    print("❌ No RealSense device detected!")
    print("   Make sure your D435 is connected via USB 3.0")
else:
    for i, dev in enumerate(devices):
        print(f"✅ Device {i}: {dev.get_info(rs.camera_info.name)}")
        print(f"   Serial Number: {dev.get_info(rs.camera_info.serial_number)}")
        print(f"   Firmware Version: {dev.get_info(rs.camera_info.firmware_version)}")
        print(f"   USB Type: {dev.get_info(rs.camera_info.usb_type_descriptor)}")

✅ Device 0: Intel RealSense D435
   Serial Number: 236522071516
   Firmware Version: 5.16.0.1
   USB Type: 3.2


## 4. Initialize Camera Pipeline

Configure and start the RealSense camera pipeline for color and depth streams at 640x480 resolution.

In [9]:
# Initialize RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()

# Configure streams - using 640x480 for good balance of speed and quality
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Start the pipeline
profile = pipeline.start(config)

print("✅ Camera pipeline started!")
print(f"   Color Stream: 640x480 @ 30fps")
print(f"   Depth Stream: 640x480 @ 30fps")

✅ Camera pipeline started!
   Color Stream: 640x480 @ 30fps
   Depth Stream: 640x480 @ 30fps


## 5. Load YOLOv8 COCO Model

Load the YOLOv8 COCO model (e.g., yolov8n.pt) and move it to the appropriate device (CPU/GPU).

In [10]:
# Load YOLOv8 COCO model (will download automatically on first run)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
coco_model = YOLO('yolov8n.pt')
coco_model.to(device)

print("✅ COCO YOLOv8 model loaded!")
print(f"   Model: YOLOv8 Nano (COCO)")
print(f"   Device: {device.upper()}" + (f" ({torch.cuda.get_device_name(0)})" if device == 'cuda' else ""))
print(f"   Classes: {len(coco_model.names)} object types")

✅ COCO YOLOv8 model loaded!
   Model: YOLOv8 Nano (COCO)
   Device: CUDA (NVIDIA GeForce RTX 5070 Ti)
   Classes: 80 object types


## 6. Real-Time Detection Loop

Run real-time detection using the YOLOv8 COCO model. Press 'q' in the OpenCV window to quit. FPS, class name, and depth are displayed for each detected object.

In [11]:
import os

def run_coco_realtime_detection(model, duration_seconds=60, record_video=True, video_path=None):
    """
    Run real-time detection using the COCO-pretrained YOLO model.
    Press 'q' to quit early. Optionally records the detection video.
    """
    print("🎥 Starting real-time detection with COCO model...")
    print("   Press 'q' in the video window to quit")
    
    start_time = time.time()
    frame_count = 0
    fps = 0
    video_writer = None
    if record_video:
        # Create 'videos' directory if it doesn't exist
        videos_dir = "videos"
        os.makedirs(videos_dir, exist_ok=True)
        # Find next available filename (01..., 02..., etc.)
        existing = [f for f in os.listdir(videos_dir) if f.endswith('.avi') and f[:2].isdigit()]
        nums = [int(f[:2]) for f in existing if f[:2].isdigit()]
        next_num = max(nums) + 1 if nums else 1
        filename = f"{next_num:02d}_detection.avi"
        video_path = os.path.join(videos_dir, filename)
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        video_writer = cv2.VideoWriter(video_path, fourcc, 30, (640, 480))
        print(f"🔴 Recording video to: {video_path}")
    try:
        while True:
            elapsed = time.time() - start_time
            if elapsed > duration_seconds:
                print(f"\n⏱️ Stopped after {duration_seconds} seconds")
                break
            frames = pipeline.wait_for_frames()
            color_frame = frames.get_color_frame()
            depth_frame = frames.get_depth_frame()
            if not color_frame:
                continue
            color_image = np.asanyarray(color_frame.get_data())
            depth_image = np.asanyarray(depth_frame.get_data())
            results = model(color_image, conf=0.3, verbose=False)
            annotated_frame = results[0].plot()
            frame_count += 1
            if frame_count % 10 == 0:
                fps = frame_count / elapsed
            cv2.putText(annotated_frame, f"FPS: {fps:.1f}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(annotated_frame, "COCO YOLOv8 Model", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
            for r in results:
                for box in r.boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
                    if 0 <= cx < 640 and 0 <= cy < 480:
                        depth_m = depth_image[cy, cx] / 1000.0
                        cv2.putText(annotated_frame, f"{depth_m:.2f}m", 
                                    (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                                    0.5, (0, 255, 255), 2)
            cv2.imshow("COCO YOLOv8 Detection - RealSense D435", annotated_frame)
            if record_video and video_writer is not None:
                video_writer.write(annotated_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                print("\n👋 Quit by user")
                break
    finally:
        cv2.destroyAllWindows()
        if record_video and video_writer is not None:
            video_writer.release()
            print(f"✅ Video saved to: {video_path}")
        print(f"✅ Detection stopped. Processed {frame_count} frames, avg FPS: {fps:.1f}")

# Run for 60 seconds (or press 'q' to quit), recording video with unique filename in 'videos' folder
run_coco_realtime_detection(coco_model, duration_seconds=60, record_video=True)


🎥 Starting real-time detection with COCO model...
   Press 'q' in the video window to quit
🔴 Recording video to: videos\04_detection.avi

⏱️ Stopped after 60 seconds
✅ Video saved to: videos\04_detection.avi
✅ Detection stopped. Processed 1800 frames, avg FPS: 30.0


## 7. Cleanup

Stop the camera pipeline and close all OpenCV windows to release resources.

In [7]:
# Stop the pipeline and release resources
if 'pipeline' in globals():
    try:
        pipeline.stop()
    except RuntimeError as e:
        print(f"⚠️ Pipeline stop skipped: {e}")
cv2.destroyAllWindows()
print("✅ Camera pipeline stopped and resources released.")

✅ Camera pipeline stopped and resources released.


# 📸 Collect Custom Images for Robotic Arm Dataset

Use the following cell to capture and save images of robotic arms with your RealSense camera. These images can be annotated and used to fine-tune your YOLOv8 model for better recognition of robot arms.

In [12]:
import os
import cv2
import pyrealsense2 as rs
import numpy as np
from datetime import datetime

# Directory to save images
save_dir = "robot_arm_dataset/images"
os.makedirs(save_dir, exist_ok=True)

# Start RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
profile = pipeline.start(config)

print("Press 's' to save an image, 'q' to quit.")
count = 0
try:
    while True:
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue
        color_image = np.asanyarray(color_frame.get_data())
        cv2.imshow('RealSense - Image Collection', color_image)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('s'):
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f"robot_arm_{timestamp}_{count:03d}.jpg"
            filepath = os.path.join(save_dir, filename)
            cv2.imwrite(filepath, color_image)
            print(f"Saved: {filepath}")
            count += 1
        elif key == ord('q'):
            print("Quitting image collection.")
            break
finally:
    pipeline.stop()
    cv2.destroyAllWindows()


Press 's' to save an image, 'q' to quit.
Saved: robot_arm_dataset/images\robot_arm_20260203_143434_000.jpg
Saved: robot_arm_dataset/images\robot_arm_20260203_143436_001.jpg
Saved: robot_arm_dataset/images\robot_arm_20260203_143455_002.jpg
Saved: robot_arm_dataset/images\robot_arm_20260203_143456_003.jpg
Saved: robot_arm_dataset/images\robot_arm_20260203_143508_004.jpg
Saved: robot_arm_dataset/images\robot_arm_20260203_143509_005.jpg
Saved: robot_arm_dataset/images\robot_arm_20260203_143543_006.jpg
Saved: robot_arm_dataset/images\robot_arm_20260203_143544_007.jpg
Saved: robot_arm_dataset/images\robot_arm_20260203_143548_008.jpg
Saved: robot_arm_dataset/images\robot_arm_20260203_143552_009.jpg
Saved: robot_arm_dataset/images\robot_arm_20260203_143554_010.jpg
Saved: robot_arm_dataset/images\robot_arm_20260203_143556_011.jpg
Saved: robot_arm_dataset/images\robot_arm_20260203_143558_012.jpg
Saved: robot_arm_dataset/images\robot_arm_20260203_143600_013.jpg
Saved: robot_arm_dataset/images\rob

## 🏷️ Annotate Images with LabelImg

Follow these steps to annotate your captured images using LabelImg:

In [ ]:
# Install LabelImg (run this cell)
%pip install labelImg

### How to Annotate with LabelImg

1. Run the previous cell to install LabelImg.
2. In your terminal, launch LabelImg:
   
   ```sh
   labelImg
   ```
3. Open your image folder (`robot_arm_dataset/images`).
4. Set the save directory to `robot_arm_dataset/labels`.
5. In the dropdown, select YOLO format.
6. Draw bounding boxes around each robotic arm and label as `robot arm`.
7. Save each annotation (a `.txt` file will be created for each image).

Once done, you’ll have YOLO-format labels ready for training!